In [0]:
r!rm -rf raw_data
!rm -rf position_decoder.h5
!rm -rf logs
!rm -rf models
# !rm -rf "../models"
# !cp -r "../models" "./"

In [0]:
pip install -e git+https://github.com/CYHSM/DeepInsight.git#egg=DeepInsight

In [0]:
# update tensorflow For a specific version:
!pip install tensorflow==1.15.2

     |████████████████████████████████| 110.5MB 106kB/s 
     |████████████████████████████████| 512kB 42.7MB/s 
     |████████████████████████████████| 3.8MB 35.8MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=0ed9deb3fe4ad71dcc3eda35d22e22a721654b18e44bf5648726d44c42370688
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: deepinsight 0.5 has requirement tensorflow==1.12.2, but you'll have tensorflow 1.15.2 which is incompatible.
  Found existing installation: tensorflow-estimator 2.2.0rc0
    Uninstalling tensorflow-estimator-2.2.0rc0:
      Successfully uninstalled tensorflow-estimator-2.2.0rc0
  Found existing installation: tensorboard 2.2.0
    Uninstalling tensorboard-2.2.0:
      Successfully uninstalled tensorboard-2.2.0
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing inst

In [0]:
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__) 
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

TensorFlow is already loaded. Please restart the runtime to change versions.
1.15.2


SystemError: ignored

In [0]:
# Import DeepInsight
import deepinsight
# Choose GPU
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import h5py

In [0]:
# create output (position)
T=40
fs_output = 30
dt=1/fs_output
f=0.15;
t=np.arange(0,T,dt)
# position=100*(1-np.cos(2*np.pi*f*t))
position=(sp.signal.sawtooth(2*np.pi*f*t)+1)*100
plt.plot(t,position,'.-')
plt.xlabel('Time (s)')
plt.ylabel('position (m/s)')
output = position
output_timestemps = t

In [0]:
# create input (spikes)
nCells=16
nPosBins = 1000;
field_width=200/nCells
maxFR = 40;
xpos=np.linspace(0,200,nPosBins)
cells_field_locs=np.linspace(0,200,nCells)
cells_spike_size=np.linspace(100,1000,nCells)
fs_raw=30000
dt2=1/fs_raw
t2=np.arange(0,T,dt2)
maps=np.zeros((nCells,len(xpos)))
# raw_data=np.zeros((nCells,len(t2)))
raw_data=np.zeros((1,len(t2)))
spike_wvf=np.array( [0,10,20,40,80,160,320,400,350,230,180,130,100,70,30,0,-30,-60,-90,-60,-30,0] )
spike_wvf=spike_wvf / max(spike_wvf)
position2=np.interp(t2,t,position)
# noise=np.random.normal(0,10,len(position2))
# noise[abs(noise<1)]=3
# position2=position2+noise
fp_hdf_raw_data = 'raw_data'
hdf5_file = h5py.File(fp_hdf_raw_data, mode='a')
hdf5_file.create_dataset("data", [len(t2),nCells], np.float16)
np.random.seed(0)
for ii_cell,mu in enumerate(cells_field_locs):
  # first, create FR map
  print(ii_cell,mu)
  FR_map=sp.stats.norm(mu, field_width).pdf(xpos)
  FR_map=FR_map/max(FR_map)*maxFR
  maps[ii_cell,:]=FR_map
  # now, apply the FR with the position variable
  FR=np.interp(position2,xpos,FR_map);
  # FRs[ii_cell,:]=FR
  spikes=np.random.poisson(FR*dt2)
  spikes=np.convolve(spikes,spike_wvf*cells_spike_size[ii_cell],mode='same')
  # raw_data[ii_cell,:]=spikes
  raw_data=raw_data+spikes
  # write data to hdf5 file
  hdf5_file["data"][:,ii_cell] = spikes
  hdf5_file.flush()
hdf5_file.close()
np.shape(raw_data)

In [0]:
hdf5_file = h5py.File(fp_hdf_raw_data, mode='a')
if not dataset_name in hdf5_file:
  hdf5_file.create_dataset(dataset_name, dataset_shape, dataset_type)  
hdf5_file.create_dataset("data2", [len(t2),nCells/2], np.float16)
hdf5_file["data2"] = np.reshape(np.array(hdf5_file["data"]),(len(t2),nCells/2,2))
hdf5_file.flush()
hdf5_file.close()

In [0]:
plt.plot(maps.transpose())
plt.xlabel('position')
plt.ylabel('FR (Hz)')
plt.title('FR maps')
plt.show()

In [0]:
# plt.plot(raw_data[::2,0*fs_raw:20*fs_raw].transpose())
plt.plot(raw_data.sum(axis=0)[0*fs_raw:20*fs_raw],color='gray')
plt.show()

In [0]:
# merge all channels to one single channel! (multiplexing...)
inputs=raw_data.sum(axis=0)[:, np.newaxis]
plt.plot(inputs)

In [0]:
import numpy.matlib
inputs=np.matlib.repmat(raw_data,2,1).transpose()

In [0]:
# fp_deepinsight = '/content/position_decoder.h5'
fp_deepinsight = './position_decoder.h5'
ds=1000

In [0]:
# preprocess input - Transform raw data to frequency domain
deepinsight.preprocess.preprocess_input(fp_deepinsight, inputs, sampling_rate=fs_raw, average_window=ds, num_cores=20)

# hdf5_file = h5py.File(fp_hdf_raw_data, mode='r')
# deepinsight.preprocess.preprocess_input(fp_deepinsight, hdf5_file['data'], sampling_rate=fs_raw, average_window=ds, num_cores=20)
# hdf5_file.close()

In [0]:
def my_create_or_update(hdf5_file, dataset_name, dataset_shape, dataset_type, dataset_value):
    """
    Create or update dataset in HDF5 file

    Parameters
    ----------
    hdf5_file : File
        File identifier
    dataset_name : str
        Name of new dataset
    dataset_shape : array_like
        Shape of new dataset
    dataset_type : type
        Type of dataset (np.float16, np.float32, 'S', etc...)
    dataset_value : array_like
        Data to store in HDF5 file
    """
    if not dataset_name in hdf5_file:
        hdf5_file.create_dataset(dataset_name, dataset_shape, dataset_type)
        hdf5_file[dataset_name][:] = dataset_value
    else:
        hdf5_file[dataset_name][:] = dataset_value
    hdf5_file.flush()

In [0]:
# preprocess outputs - align output timestemps
hdf5_file = h5py.File(fp_deepinsight, mode='a')
# Get size of wavelets
input_length = hdf5_file['inputs/wavelets'].shape[0]
# downsample output
position3 = np.interp(t2[np.arange(0,t2.shape[0],ds)], t, position)
# noise=np.random.normal(0,10,len(position3))
# noise[abs(noise<1)]=3
# position3=position3+noise
# Create and save datasets in HDF5 File
my_create_or_update(hdf5_file, dataset_name="outputs/position", 
                      dataset_shape=[input_length, 1], dataset_type=np.float16, dataset_value=position3[0: input_length, np.newaxis])

hdf5_file.flush()
hdf5_file.close()

In [0]:
 # check inputs and outputs!
hdf5_file = h5py.File(fp_deepinsight, mode='r')
wvlts = hdf5_file['inputs/wavelets']
ff = hdf5_file['inputs/fourier_frequencies']
position_out = hdf5_file['outputs/position']
print(wvlts)
print(ff)
print(position_out)
plt.subplot(3,1,1)
plt.plot(position_out[:],'.',color="blue")
plt.subplot(3,1,2)
plt.plot(wvlts[:,:,1])
plt.subplot(3,1,3)
plt.imshow(wvlts[:,:,0].transpose(),aspect='auto')
# plt.colorbar()
# plt.plot(np.mean(wvlts,axis=1)[:700,:])
# print(np.shape(np.mean(wvlts,axis=1)))

# plt.plot(ff,'.-')
print(ff[7])
plt.show()
hdf5_file.close()
np.shape(t2)

In [0]:
def myRun(path_in, loss_functions, loss_weights, num_cvs=5):
    """
    Runs model training giving path to HDF5 file and loss dictionaries
    Parameters
    ----------
    path_in : str
        Path to HDF5
    loss_functions : dict
        For each output the corresponding loss function
    loss_weights : dict
        For each output the corresponding weight
    num_cvs : int, optional
        Number of cross validation splits, by default 5
    """
    dirname = os.path.dirname(path_in)
    filename = os.path.basename(path_in)
    # Define folders
    tensorboard_logfolder = dirname + '/logs/' + filename[0:-3]  # Remove .h5 for logfolder
    model_tmp_path = dirname + '/models/tmp/tmp_model'
    model_path = dirname + '/models/' + filename[0:-3] + '_model.h5'
    # Create folders if needed
    for f in [os.path.dirname(model_tmp_path), os.path.dirname(model_path)]:
        if not os.path.exists(f):
            os.makedirs(f)
    print('------------------------------------------------')
    print('-> Running {} from {}'.format(filename, dirname))
    print('- Logs : {} \n- Model temporary : {} \n- Model : {}'.format(tensorboard_logfolder, model_tmp_path, model_path))
    print('------------------------------------------------')
    # Train model
    print('------------------------------------------------')
    print('Starting standard model')
    print('------------------------------------------------')
    deepinsight.train.train_model(model_path, path_in, tensorboard_logfolder, model_tmp_path, loss_functions, loss_weights, num_cvs)


In [0]:
# RUN!!!

# Define loss functions and train model
loss_functions = {'position' : 'euclidean_loss'}
loss_weights = {'position' : 1}

# deepinsight.train.run_from_path(fp_deepinsight, loss_functions, loss_weights)
with tf.device('/device:GPU:0'):
  myRun(fp_deepinsight, loss_functions, loss_weights, num_cvs=2)
  # deepinsight.train.run_from_path(fp_deepinsight, loss_functions, loss_weights)

In [0]:
losses, output_predictions, indices = deepinsight.analyse.get_model_loss(fp_deepinsight,stepsize=1)
shuffled_losses = deepinsight.analyse.get_shuffled_model_loss(fp_deepinsight, axis=1,stepsize=1)

In [0]:
plt.subplot(2,1,1)
plt.plot(position3)
plt.plot(indices,output_predictions['position'])
plt.legend(['original','predicted'])
plt.subplot(2,1,2)
xxx1=0
xxx2=100
plt.plot(indices[xxx1:xxx2],position3[xxx1:xxx2])
plt.plot(indices[xxx1:xxx2],output_predictions['position'][xxx1:xxx2])
plt.legend(['original','predicted'])
plt.savefig('position_decoding.pdf')

In [0]:
output_predictions['position'][:64].transpose()

In [0]:
plt.plot(position3, output_predictions['position'], '.')
# plt.plot(position3[xxx1:xxx2], output_predictions['position'][xxx1:xxx2], '.')

In [0]:
output_predictions['position'][:66].transpose()

In [0]:
plt.plot(losses)
plt.plot(shuffled_losses[:,:,0].transpose())

In [0]:
np.diff(indices)